<a href="https://colab.research.google.com/github/alhe/Bias_mitigation/blob/main/UPP26/TD2_mepsdata_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MiProjet Bernabé Clarence



## Introduction
Pour ce projet, nous allons analyser un sous-ensemble des métasdonnées de 15000 individus, issus du jeu de données Chest X ray NIH 14 Dataset : https://www.kaggle.com/datasets/nih-chest-xrays/data. Le fichier contenant les données de départs est Bernabe_Clarence.csv.

### Objectifs:
1. Analyse des métadonnées
2. Identification de biais (s'il en existe)
3. Réduction des biais grâce à une méthode de pré-processing (nouveau dataset avec biais réduits)

### Présentation du dataset
Ce dataset contient 53467 lignes,

In [1]:
import numpy as np
#import fairlearn
import plotly.express as px
import pandas as pd
#np.__version__
#fairlearn.__version__
print("Importation des bibliothèques réussi.\n")

Importation des bibliothèques réussi.



In [2]:
df = pd.read_csv("Bernabe_Clarence.csv")
print("Chargement du dataset sous forme dataframe réussi.\n")

NameError: name 'pd' is not defined

In [ ]:
df.iloc[list(range(5)) + list(range(-5, 0))]


In [ ]:
df["Patient ID"].nunique()

15000